## Model training and tuning for the utilities sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.models.forecasting.sf_auto_arima import AutoARIMA
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_utilities = df[df['GICS Sector'] == 'Utilities'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [3]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [4]:
df_utilities['ticker'].unique()

array(['AEE', 'AEP', 'AES', 'ATO', 'AWK', 'CEG', 'CMS', 'CNP', 'D', 'DTE',
       'DUK', 'ED', 'EIX', 'ES', 'ETR', 'EVRG', 'EXC', 'FE', 'LNT', 'NEE',
       'NI', 'NRG', 'PCG', 'PEG', 'PNW', 'PPL', 'SO', 'SRE', 'VST', 'WEC',
       'XEL'], dtype=object)

In [5]:
data = {}
static_covs = pd.DataFrame({'ticker': df_utilities['ticker'].unique()})

for ticker, group in df_utilities.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [6]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [7]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [8]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for AEE
Model for AEE trained in 0.31 seconds
**************************************************
Training model for AEP
Model for AEP trained in 0.34 seconds
**************************************************
Training model for AES
Model for AES trained in 0.33 seconds
**************************************************
Training model for ATO
Model for ATO trained in 0.31 seconds
**************************************************
Training model for AWK
Model for AWK trained in 0.33 seconds
**************************************************
Training model for CEG
Model for CEG trained in 0.23 seconds
**************************************************
Training model for CMS
Model for CMS trained in 0.31 seconds
**************************************************
Training model for CNP
Model for CNP trained in 0.36 seconds
**************************************************
Training model for D
Model for D trained in 0.38 seconds
***********************************************

In [9]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AEE, MAPE: 8.16, MAE: 7.33
**************************************************
Ticker: AEP, MAPE: 7.92, MAE: 7.73
**************************************************
Ticker: AES, MAPE: 5.73, MAE: 0.82
**************************************************
Ticker: ATO, MAPE: 14.86, MAE: 21.27
**************************************************
Ticker: AWK, MAPE: 4.48, MAE: 6.23
**************************************************
Ticker: CEG, MAPE: 9.53, MAE: 26.43
**************************************************
Ticker: CMS, MAPE: 19.86, MAE: 13.53
**************************************************
Ticker: CNP, MAPE: 9.84, MAE: 3.29
**************************************************
Ticker: D, MAPE: 2.66, MAE: 1.45
**************************************************
Ticker: DTE, MAPE: 8.87, MAE: 10.93
**************************************************
Ticker: DUK, MAPE: 9.55, MAE: 10.88
**************************************************
Ticker: ED, MAPE: 7.36, MAE: 7.54
***************

In [10]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(10.883061202967538, 9.540799941432756)

### Global model

In [11]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 12.09 seconds
**************************************************


In [12]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for AEE, MAPE: 2.52, MAE: 2.15
**************************************************
Global model for AEP, MAPE: 4.63, MAE: 4.66
**************************************************
Global model for AES, MAPE: 17.13, MAE: 1.97
**************************************************
Global model for ATO, MAPE: 4.37, MAE: 6.07
**************************************************
Global model for AWK, MAPE: 9.37, MAE: 12.74
**************************************************
Global model for CEG, MAPE: 14.32, MAE: 40.44
**************************************************
Global model for CMS, MAPE: 4.77, MAE: 3.27
**************************************************
Global model for CNP, MAPE: 11.72, MAE: 4.01
**************************************************
Global model for D, MAPE: 6.11, MAE: 3.27
**************************************************
Global model for DTE, MAPE: 3.10, MAE: 3.91
**************************************************
Global model for DUK, MAPE: 8.34, MAE: 9.61
***

In [13]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(8.542079555821216, 6.87950003636237)

## Prophet model

In [14]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for AEE


22:04:43 - cmdstanpy - INFO - Chain [1] start processing
22:04:46 - cmdstanpy - INFO - Chain [1] done processing


Model for AEE trained in 3.12 seconds
**************************************************
Training model for AEP


22:04:46 - cmdstanpy - INFO - Chain [1] start processing
22:04:49 - cmdstanpy - INFO - Chain [1] done processing


Model for AEP trained in 3.35 seconds
**************************************************
Training model for AES


...

Model for VST trained in 1.95 seconds
**************************************************
Training model for WEC


22:06:03 - cmdstanpy - INFO - Chain [1] start processing
22:06:06 - cmdstanpy - INFO - Chain [1] done processing


Model for WEC trained in 3.25 seconds
**************************************************
Training model for XEL


22:06:07 - cmdstanpy - INFO - Chain [1] start processing
22:06:09 - cmdstanpy - INFO - Chain [1] done processing


Model for XEL trained in 3.24 seconds
**************************************************


In [15]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AEE, MAPE: 32.84, MAE: 29.75
**************************************************
Ticker: AEP, MAPE: 29.55, MAE: 28.88
**************************************************
Ticker: AES, MAPE: 98.23, MAE: 13.65
**************************************************
Ticker: ATO, MAPE: 18.22, MAE: 25.99
**************************************************
Ticker: AWK, MAPE: 38.49, MAE: 52.46
**************************************************
Ticker: CEG, MAPE: 163.05, MAE: 351.63
**************************************************
Ticker: CMS, MAPE: 21.93, MAE: 14.95
**************************************************
Ticker: CNP, MAPE: 16.04, MAE: 5.22
**************************************************
Ticker: D, MAPE: 41.28, MAE: 22.37
**************************************************
Ticker: DTE, MAPE: 21.97, MAE: 27.16
**************************************************
Ticker: DUK, MAPE: 23.83, MAE: 26.64
**************************************************
Ticker: ED, MAPE: 16.13, MAE: 16.

In [16]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(33.94790047574618, 31.823623686722204)

## Theta model

In [17]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for AEE
Model for AEE trained in 1.06 seconds
**************************************************
Training model for AEP
Model for AEP trained in 0.61 seconds
**************************************************
Training model for AES
Model for AES trained in 0.60 seconds
**************************************************
Training model for ATO
Model for ATO trained in 1.02 seconds
**************************************************
Training model for AWK
Model for AWK trained in 0.63 seconds
**************************************************
Training model for CEG
Model for CEG trained in 0.16 seconds
**************************************************
Training model for CMS
Model for CMS trained in 0.67 seconds
**************************************************
Training model for CNP
Model for CNP trained in 0.68 seconds
**************************************************
Training model for D
Model for D trained in 1.08 seconds
***********************************************

In [18]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AEE, MAPE: 13.77, MAE: 12.53
**************************************************
Ticker: AEP, MAPE: 10.03, MAE: 10.03
**************************************************
Ticker: AES, MAPE: 34.95, MAE: 4.18
**************************************************
Ticker: ATO, MAPE: 12.90, MAE: 18.43
**************************************************
Ticker: AWK, MAPE: 7.19, MAE: 10.00
**************************************************
Ticker: CEG, MAPE: 21.05, MAE: 50.23
**************************************************
Ticker: CMS, MAPE: 9.51, MAE: 6.57
**************************************************
Ticker: CNP, MAPE: 8.20, MAE: 2.69
**************************************************
Ticker: D, MAPE: 8.70, MAE: 4.78
**************************************************
Ticker: DTE, MAPE: 9.00, MAE: 11.32
**************************************************
Ticker: DUK, MAPE: 11.42, MAE: 12.94
**************************************************
Ticker: ED, MAPE: 6.29, MAE: 6.37
*********

In [19]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(12.94047385746659, 10.775684431093415)

## Linear Regression

### Local model

In [20]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for AEE
Model for AEE trained in 0.13 seconds
**************************************************
Training model for AEP
Model for AEP trained in 0.12 seconds
**************************************************
Training model for AES
Model for AES trained in 0.12 seconds
**************************************************
Training model for ATO
Model for ATO trained in 0.13 seconds
**************************************************
Training model for AWK
Model for AWK trained in 0.16 seconds
**************************************************
Training model for CEG
Model for CEG trained in 0.07 seconds
**************************************************
Training model for CMS
Model for CMS trained in 0.12 seconds
**************************************************
Training model for CNP
Model for CNP trained in 0.12 seconds
**************************************************
Training model for D
Model for D trained in 0.12 seconds
***********************************************

In [21]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AEE, MAPE: 488565.07, MAE: 479821.16
**************************************************
Ticker: AEP, MAPE: 14473.51, MAE: 15211.79
**************************************************
Ticker: AES, MAPE: 9.98, MAE: 1.31
**************************************************
Ticker: ATO, MAPE: 311814.96, MAE: 482770.06
**************************************************
Ticker: AWK, MAPE: 1.45, MAE: 1.95
**************************************************
Ticker: CEG, MAPE: 4231.06, MAE: 9158.41
**************************************************
Ticker: CMS, MAPE: 2.29, MAE: 1.53
**************************************************
Ticker: CNP, MAPE: 1.68, MAE: 0.52
**************************************************
Ticker: D, MAPE: 1.46, MAE: 0.78
**************************************************
Ticker: DTE, MAPE: 2.81, MAE: 3.48
**************************************************
Ticker: DUK, MAPE: 1.93, MAE: 2.16
**************************************************
Ticker: ED, MAPE: 3045

In [22]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(150721948.77610624, 62969523.58290918)

### Global model

In [23]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 3.68 seconds
**************************************************


In [24]:
mapes_lr_global = {}
maes_lr_global = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes_lr_global[ticker] = mape_
    maes_lr_global[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for AEE, MAPE: 2.24, MAE: 1.92
**************************************************
Global model for AEP, MAPE: 2.28, MAE: 2.21
**************************************************
Global model for AES, MAPE: 6.69, MAE: 0.92
**************************************************
Global model for ATO, MAPE: 2.17, MAE: 2.98
**************************************************
Global model for AWK, MAPE: 2.52, MAE: 3.40
**************************************************
Global model for CEG, MAPE: 10.44, MAE: 25.74
**************************************************
Global model for CMS, MAPE: 2.09, MAE: 1.39
**************************************************
Global model for CNP, MAPE: 2.65, MAE: 0.80
**************************************************
Global model for D, MAPE: 2.31, MAE: 1.23
**************************************************
Global model for DTE, MAPE: 2.13, MAE: 2.60
**************************************************
Global model for DUK, MAPE: 2.15, MAE: 2.38
******

In [25]:
np.mean(list(mapes_lr_global.values())), np.mean(list(maes_lr_global.values()))

(3.124581792018007, 2.887410301610158)

## ARIMA

In [26]:
models_arima = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoARIMA()

    model.fit(
        series=train[ticker]['close'],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_arima[ticker] = model

Training model for AEE
Model for AEE trained in 4.09 seconds
**************************************************
Training model for AEP
Model for AEP trained in 0.17 seconds
**************************************************
Training model for AES
Model for AES trained in 1.02 seconds
**************************************************
Training model for ATO
Model for ATO trained in 2.94 seconds
**************************************************
Training model for AWK
Model for AWK trained in 0.18 seconds
**************************************************
Training model for CEG
Model for CEG trained in 0.52 seconds
**************************************************
Training model for CMS
Model for CMS trained in 4.55 seconds
**************************************************
Training model for CNP
Model for CNP trained in 1.33 seconds
**************************************************
Training model for D
Model for D trained in 3.73 seconds
***********************************************

In [27]:
mapes_arima = {}
maes_arima = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_arima[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_arima[ticker] = mape_
    maes_arima[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AEE, MAPE: 15.33, MAE: 13.99
**************************************************
Ticker: AEP, MAPE: 11.34, MAE: 11.33
**************************************************
Ticker: AES, MAPE: 32.02, MAE: 3.83
**************************************************
Ticker: ATO, MAPE: 11.53, MAE: 16.44
**************************************************
Ticker: AWK, MAPE: 8.50, MAE: 11.88
**************************************************
Ticker: CEG, MAPE: 21.81, MAE: 51.82
**************************************************
Ticker: CMS, MAPE: 12.05, MAE: 8.32
**************************************************
Ticker: CNP, MAPE: 8.91, MAE: 2.93
**************************************************
Ticker: D, MAPE: 9.33, MAE: 5.13
**************************************************
Ticker: DTE, MAPE: 10.34, MAE: 12.99
**************************************************
Ticker: DUK, MAPE: 12.78, MAE: 14.47
**************************************************
Ticker: ED, MAPE: 6.24, MAE: 6.32
*******

In [28]:
np.mean(list(mapes_arima.values())), np.mean(list(maes_arima.values()))

(13.175480744474507, 10.86896206866302)